In [16]:
#!/usr/bin/env python3
"""
Translate Excel SHEET CONTENTS (cell text) using a user-provided map.json.

- Asks user:
  1) Did you copy + paste by values in all sheets within all workbooks?
  2) Folder path containing the 3M / 12WM / 36WM workbooks.

- Does NOT rename existing sheets
- Adds a new first sheet "Sheet Name Trans." with:
  * Original sheet names
  * English translations of sheet names (using map.json)
- "Sheet Name Trans." is NOT scanned/translated like other sheets.

- Does NOT modify formulas (skips them by default)
- Does NOT overwrite map.json (read-only)
- Uses strict Hangul-with-spaces phrases + remaining Hangul runs
- Writes:
  * 3M-full_update-translated.xlsx
  * 12WM-full_update-translated.xlsx
  * 36WM-full_update-translated.xlsx
  * missing_items.csv (Hangul detected but not present in map.json)
"""

import re
import json
import collections
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import pandas as pd
from openpyxl import load_workbook

# -----------------------------
# CONFIG
# -----------------------------

# Tags we expect in filenames to identify each workbook
WORKBOOK_TAGS = ["3M", "12WM", "36WM"]

# Use your cleaned map file here (or rename it to map.json)
MAP_JSON = "map.json"

# Name for the missing items CSV (will be written into outputs/ folder)
MISSING_CSV = "missing_items.csv"

SKIP_FORMULAS = True
APPLY_SPACE_NORMALIZATION = True

SHEET_NAME_TRANSLATION_TITLE = "Sheet Name Trans."


# -----------------------------
# HANGUL DETECTION / EXTRACTION
# -----------------------------
HANGUL_CHAR_RE = re.compile(r"[\uac00-\ud7a3]")
HANGUL_PHRASE_RE = re.compile(r"[\uac00-\ud7a3]+(?:\s+[\uac00-\ud7a3]+)+")
HANGUL_RUN_RE = re.compile(r"[\uac00-\ud7a3]+")


def has_hangul(s: str) -> bool:
    return isinstance(s, str) and bool(HANGUL_CHAR_RE.search(s))


def extract_phrases_and_runs_strict(text: str) -> Tuple[List[str], List[str]]:
    """
    Extract:
      - phrases: Hangul words separated by spaces
      - runs: remaining contiguous Hangul sequences after removing phrases
    """
    phrases = HANGUL_PHRASE_RE.findall(text)
    remainder = text
    for p in phrases:
        remainder = remainder.replace(p, " ")
    runs = HANGUL_RUN_RE.findall(remainder)
    return phrases, runs


# -----------------------------
# OPTIONAL SPACE NORMALIZATION
# -----------------------------
def normalize_spaces(text: str) -> str:
    if not isinstance(text, str):
        return text
    # ) followed by alnum: add space
    text = re.sub(r"\)(?=[A-Za-z0-9])", ") ", text)
    # Alpha right next to Hangul (both directions): add space
    text = re.sub(r"([A-Za-z])([\uac00-\ud7a3])", r"\1 \2", text)
    text = re.sub(r"([\uac00-\ud7a3])([A-Za-z])", r"\1 \2", text)
    # Hangul right before "Plant" (with optional digits): add space
    text = re.sub(r"([\uac00-\ud7a3])(\d*Plant)", r"\1 \2", text)
    # Collapse multiple spaces
    text = re.sub(r"\s{2,}", " ", text).strip()
    return text


# -----------------------------
# TOKEN PROTECTION
# -----------------------------
# Protect codes/ids/numbers; protect parentheses only if NO Hangul inside.
TOKEN_PATTERNS = [
    r"\[[^\]]+\]",                           # [ABC123]
    r"\b[A-Z]{1,5}\d{0,5}\b",                # codes like ABC12
    r"\b\d+(?:\.\d+)?(?:E[-+]\d+)?\b",       # numbers, floats, scientific notation
    r"\((?!.*[\uac00-\ud7a3])[^\)]*\)",      # (...) with no Hangul inside
]
TOKEN_RE = re.compile("|".join(f"({p})" for p in TOKEN_PATTERNS))


def protect_tokens(text: str) -> Tuple[str, List[str]]:
    tokens: List[str] = []

    def repl(m):
        tokens.append(m.group(0))
        return f"__TOK{len(tokens)-1}__"

    return TOKEN_RE.sub(repl, text), tokens


def restore_tokens(text: str, tokens: List[str]) -> str:
    for i, tok in enumerate(tokens):
        text = text.replace(f"__TOK{i}__", tok)
    return text


# -----------------------------
# MAP LOADING (READ-ONLY)
# -----------------------------
def load_fragments_map(path: str) -> Dict[str, str]:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Map file not found: {p.resolve()}")
    with p.open("r", encoding="utf-8") as f:
        data = json.load(f)
    fragments = data.get("fragments", {})
    if not isinstance(fragments, dict):
        raise ValueError('Map must look like: {"fragments": { ... }}')
    return fragments


def build_pairs(fragments: Dict[str, str]) -> List[Tuple[str, str]]:
    # Sort keys longest-first to avoid partial-replacement collisions
    keys = sorted(fragments.keys(), key=len, reverse=True)
    return [(k, fragments[k]) for k in keys]


def translate_text_with_map(text: str, pairs: List[Tuple[str, str]]) -> str:
    masked, toks = protect_tokens(text)
    out = masked
    for k, v in pairs:
        out = out.replace(k, v)
    out = restore_tokens(out, toks)
    return out


# -----------------------------
# WORKBOOK DISCOVERY
# -----------------------------
def find_workbooks_by_tag(folder: Path, tags: List[str]) -> Optional[Dict[str, Path]]:
    """
    Scan folder for .xlsx files and map each tag (e.g. '3M') to exactly one file
    whose name contains that tag.

    Returns:
        dict[tag -> Path] if successful, otherwise None (and prints errors).
    """
    if not folder.is_dir():
        print(f"\nERROR: Input folder not found: {folder.resolve()}")
        return None

    all_xlsx = [p for p in folder.glob("*.xlsx") if not p.name.startswith("~$")]

    print("\nSearching for workbooks in:", folder.resolve())
    if not all_xlsx:
        print("No .xlsx files found in this folder.")
    else:
        print("Found the following .xlsx files:")
        for p in all_xlsx:
            print(f"  - {p.name}")

    mapping: Dict[str, Path] = {}
    errors: List[str] = []

    print("\nTag -> matched files:")
    for tag in tags:
        matches = [p for p in all_xlsx if tag in p.name]

        if len(matches) == 0:
            msg = (
                f"  For tag '{tag}': expected 1 file, found 0. "
                f"No filenames containing '{tag}'."
            )
            print(msg)
            errors.append(msg)
        elif len(matches) == 1:
            print(f"  For tag '{tag}': OK -> {matches[0].name}")
            mapping[tag] = matches[0]
        else:
            names = ", ".join(m.name for m in matches)
            msg = (
                f"  For tag '{tag}': expected 1 file, found {len(matches)}: {names}"
            )
            print(msg)
            errors.append(msg)

    if errors:
        print("\nERROR: Workbook detection failed.")
        print("Details:")
        for e in errors:
            print(" -", e)
        print(
            "\nPlease ensure there is exactly one file containing each of "
            f"{', '.join(tags)} in the selected folder, then re-run the script."
        )
        return None

    print("\nWorkbook mapping (final):")
    for tag, path in mapping.items():
        print(f"  {tag} -> {path.name}")

    return mapping


# -----------------------------
# SHEET NAME TRANSLATION SHEET
# -----------------------------
def add_sheet_name_translation_sheet(
    wb,
    input_path: Path,
    fragments: Dict[str, str],
    pairs: List[Tuple[str, str]],
    missing_counts: collections.Counter,
    missing_example: Dict[str, str],
) -> None:
    """
    Create a new first sheet "Sheet Name Trans." listing:
      - Original sheet names
      - Translated sheet names

    NOTE: This version does NOT contribute sheet-name Hangul
    to missing_counts / missing_example. missing_items.csv
    now reflects ONLY cell contents.
    """
    # Get original sheet names (before adding the new sheet)
    original_sheet_names = list(wb.sheetnames)

    # Compute translations for sheet names (no missing-items tracking here)
    translations: List[Tuple[str, str]] = []
    for sheet_name in original_sheet_names:
        original = sheet_name
        processed = original.strip()
        if APPLY_SPACE_NORMALIZATION:
            processed = normalize_spaces(processed)

        # Translate sheet name (for display only; we do NOT rename sheets)
        translated = translate_text_with_map(processed, pairs)
        if APPLY_SPACE_NORMALIZATION:
            translated = normalize_spaces(translated)

        translations.append((original, translated))

    # Create the new sheet at position 0
    sheet = wb.create_sheet(title=SHEET_NAME_TRANSLATION_TITLE, index=0)

    # Header row
    sheet["A1"] = "Original Sheet Name"
    sheet["B1"] = "Translated Sheet Name"

    # Data rows
    row_idx = 2
    for original, translated in translations:
        sheet.cell(row=row_idx, column=1, value=original)
        sheet.cell(row=row_idx, column=2, value=translated)
        row_idx += 1
# -----------------------------
# MAIN
# -----------------------------
def main():
    # 1. Safety confirmation in terminal
    print(
        "\nIMPORTANT CHECK:\n"
        "This script is intended to run on workbooks where all formulas have been\n"
        "replaced by VALUES (copy + paste by values) to avoid breaking references.\n"
    )
    answer = input(
        "Did you copy + paste by values in all sheets within all workbooks? [y/N]: "
    ).strip().lower()

    if answer not in ("y", "yes"):
        print(
            "\nAborting: Please create value-only versions of your workbooks first, "
            "then re-run this script."
        )
        return

    # 2. Ask user for folder path
    print(
        "\nPlease enter the folder path containing the 3 workbooks "
        "(files whose names contain '3M', '12WM', and '36WM')."
    )
    folder_input = input("Folder path: ").strip()

    if not folder_input:
        print("\nNo folder path entered. Aborting.")
        return

    # Allow user to paste with quotes; strip leading/trailing quotes if present
    folder_input = folder_input.strip().strip('"').strip("'")
    input_folder = Path(folder_input)
    print(f"\nUsing input folder: {input_folder.resolve()}")

    # 3. Resolve script directory and outputs folder
    try:
        script_dir = Path(__file__).resolve().parent
    except NameError:
        # Fallback if __file__ is not defined (e.g. in some interactive environments)
        script_dir = Path.cwd()

    outputs_dir = script_dir / "outputs"
    outputs_dir.mkdir(exist_ok=True)
    print(f"Outputs will be written to: {outputs_dir.resolve()}")

    # 4. Load translation map (fragments)
    fragments = load_fragments_map(MAP_JSON)
    pairs = build_pairs(fragments)

    # 5. Discover the 3 target workbooks in the input folder
    files_by_tag = find_workbooks_by_tag(input_folder, WORKBOOK_TAGS)
    if files_by_tag is None:
        # Errors already printed; just abort
        return

    # 6. Global counters for missing items across ALL workbooks
    missing_counts = collections.Counter()
    missing_example: Dict[str, str] = {}

    total_scanned = 0
    total_changed = 0

    # 7. Process each workbook
    for tag, input_path in files_by_tag.items():
        print(f"\n⏳ Processing {tag} workbook: {input_path.name}")

        wb = load_workbook(input_path, data_only=False)

        # 7a. Add "Sheet Name Trans." sheet with sheet-name translations
        add_sheet_name_translation_sheet(
            wb, input_path, fragments, pairs, missing_counts, missing_example
        )

        scanned = 0
        changed = 0

        # 7b. Translate cell contents on all sheets EXCEPT "Sheet Name Trans."
        for ws in wb.worksheets:
            if ws.title == SHEET_NAME_TRANSLATION_TITLE:
                # Skip the translation sheet itself
                continue

            for row in ws.iter_rows():
                for cell in row:
                    v = cell.value
                    if not isinstance(v, str):
                        continue
                    if SKIP_FORMULAS and v.startswith("="):
                        continue

                    scanned += 1
                    text = v.strip()
                    if APPLY_SPACE_NORMALIZATION:
                        text = normalize_spaces(text)

                    if not has_hangul(text):
                        # Still write normalized text (e.g. spacing changes) if needed
                        if APPLY_SPACE_NORMALIZATION and text != v:
                            cell.value = text
                        continue

                    # Missing Hangul detection for cell contents
                    phrases, runs = extract_phrases_and_runs_strict(text)
                    for item in phrases + runs:
                        if item and item not in fragments:
                            missing_counts[item] += 1
                            example_key = (
                                f"{input_path.name}:{ws.title}!{cell.coordinate}"
                            )
                            missing_example.setdefault(item, example_key)

                    # Translation
                    out = translate_text_with_map(text, pairs)
                    if APPLY_SPACE_NORMALIZATION:
                        out = normalize_spaces(out)

                    if out != v:
                        cell.value = out
                        changed += 1

        # 7c. Save translated workbook for this tag
        output_name = f"{tag}-full_update-translated.xlsx"
        output_path = outputs_dir / output_name
        wb.save(output_path)

        print(f"✅ Finished processing {tag}.")
        print(f"  Output workbook: {output_path.name}")
        print(f"  Scanned text cells (non-formula): {scanned}")
        print(f"  Changed cells: {changed}")

        total_scanned += scanned
        total_changed += changed

    # 8. Write combined missing_items.csv for all workbooks
    df_missing = pd.DataFrame(
        [
            {
                "hangul": k,
                "count": int(c),
                "example_cell": missing_example.get(k, ""),
            }
            for k, c in missing_counts.most_common()
        ]
    )

    missing_csv_path = outputs_dir / MISSING_CSV
    df_missing.to_csv(missing_csv_path, index=False, encoding="utf-8-sig")

    print("\n=== All workbooks processed ===")
    print(f"Total scanned text cells (non-formula): {total_scanned}")
    print(f"Total changed cells: {total_changed}")
    print(
        f"Missing unique Hangul items across all workbooks: {len(missing_counts)} "
        f"-> {missing_csv_path.name}"
    )


if __name__ == "__main__":
    main()


IMPORTANT CHECK:
This script is intended to run on workbooks where all formulas have been
replaced by VALUES (copy + paste by values) to avoid breaking references.


Please enter the folder path containing the 3 workbooks (files whose names contain '3M', '12WM', and '36WM').

Using input folder: C:\Users\r103317\Projects\Monthly Report\monthly-reporter\upload
Outputs will be written to: C:\Users\r103317\Projects\Monthly Report\monthly-reporter\outputs

Searching for workbooks in: C:\Users\r103317\Projects\Monthly Report\monthly-reporter\upload
Found the following .xlsx files:
  - 26.1-12WM_update.xlsx
  - 26.1-36WM_update.xlsx
  - 26.1-3M_update.xlsx

Tag -> matched files:
  For tag '3M': OK -> 26.1-3M_update.xlsx
  For tag '12WM': OK -> 26.1-12WM_update.xlsx
  For tag '36WM': OK -> 26.1-36WM_update.xlsx

Workbook mapping (final):
  3M -> 26.1-3M_update.xlsx
  12WM -> 26.1-12WM_update.xlsx
  36WM -> 26.1-36WM_update.xlsx

⏳ Processing 3M workbook: 26.1-3M_update.xlsx
✅ Finished proces